# Large-Scale Point Visualization with Datashader

This notebook demonstrates large-scale point rendering using the datashader integration adapter.
It showcases zero-copy data paths and overlay texture generation for millions of points.

**Expected runtime:** < 8 minutes  
**Memory usage:** < 400 MiB  
**Outputs:** datashader_points.png

In [ ]:
import numpy as np
import sys
import os
import time
from pathlib import Path
import json

# Add repo root to path for imports
if Path('../..').exists():
    sys.path.insert(0, str(Path('../..').resolve()))

try:
    import forge3d as f3d
    print(f"✓ forge3d {f3d.__version__} loaded successfully")
except ImportError as e:
    print(f"✗ Failed to import forge3d: {e}")
    print("Please run: maturin develop --release")
    sys.exit(1)

# Check datashader availability
try:
    from forge3d.adapters import (
        DatashaderAdapter, 
        is_datashader_available,
        rgba_view_from_agg,
        validate_alignment,
        shade_to_overlay
    )
    
    datashader_available = is_datashader_available()
    print(f"✓ Datashader integration available: {datashader_available}")
    
    if datashader_available:
        import datashader as ds
        import datashader.transfer_functions as tf
        import pandas as pd
        print(f"✓ datashader {ds.__version__} ready")
    else:
        print("⚠ datashader not available - some features will be skipped")
        
except ImportError as e:
    print(f"⚠ Datashader adapter import failed: {e}")
    datashader_available = False

## Device Setup & Memory Budget

Configure GPU device with memory budget awareness for large-scale data processing.

In [ ]:
# Device information and memory budget
try:
    device_info = f3d.device_probe()
    print("🖥️  Device Configuration:")
    print(f"   Backend: {device_info.get('backend', 'unknown')}")
    print(f"   Device: {device_info.get('adapter_name', 'unknown')}")
    
    # Memory budget for large-scale processing
    memory_budget_mb = 400  # Stay well below 512MB limit
    print(f"   Memory budget: {memory_budget_mb} MB (target <512 MB)")
    
except Exception as e:
    print(f"⚠ Device probe failed: {e}")
    memory_budget_mb = 256  # Conservative fallback

## Large-Scale Point Data Generation

Generate synthetic point cloud data representing a realistic spatial distribution.

In [ ]:
# Generate large point dataset
np.random.seed(42)  # Deterministic results

# Scale based on memory budget (roughly 40 bytes per point with overhead)
max_points = int((memory_budget_mb * 1024 * 1024) / 40)
num_points = min(1_000_000, max_points)  # Cap at 1M for demo

print(f"📊 Generating point cloud: {num_points:,} points")

start_time = time.time()

# Create realistic spatial distribution
# Mixture of clustered and uniform distributions
cluster_fraction = 0.7
num_clusters = int(num_points * cluster_fraction)
num_uniform = num_points - num_clusters

# Clustered points (cities, hotspots)
cluster_centers = np.random.uniform(-100, 100, (5, 2))  # 5 cluster centers
cluster_points = []
points_per_cluster = num_clusters // len(cluster_centers)

for center in cluster_centers:
    # Normal distribution around cluster center
    cluster_x = np.random.normal(center[0], 15, points_per_cluster)
    cluster_y = np.random.normal(center[1], 15, points_per_cluster)
    cluster_points.extend(list(zip(cluster_x, cluster_y)))

# Uniform background points
uniform_x = np.random.uniform(-200, 200, num_uniform)
uniform_y = np.random.uniform(-200, 200, num_uniform)
uniform_points = list(zip(uniform_x, uniform_y))

# Combine all points
all_points = cluster_points + uniform_points
np.random.shuffle(all_points)  # Mix cluster and uniform points

# Convert to arrays
points_array = np.array(all_points[:num_points], dtype=np.float32)
x_coords = points_array[:, 0]
y_coords = points_array[:, 1]

# Add value dimension for coloring
values = np.random.exponential(scale=2.0, size=num_points).astype(np.float32)
values = np.clip(values, 0, 10)  # Reasonable range for visualization

generation_time = (time.time() - start_time) * 1000

print(f"✓ Point generation complete: {generation_time:.1f} ms")
print(f"   X range: [{x_coords.min():.1f}, {x_coords.max():.1f}]")
print(f"   Y range: [{y_coords.min():.1f}, {y_coords.max():.1f}]")
print(f"   Value range: [{values.min():.2f}, {values.max():.2f}]")
print(f"   Memory: {points_array.nbytes / 1024 / 1024:.1f} MB")

## Datashader Aggregation

Use datashader to aggregate points into a raster for GPU rendering.

In [ ]:
if datashader_available:
    print("🎯 Datashader aggregation...")
    
    # Create DataFrame for datashader
    df_start = time.time()
    df = pd.DataFrame({
        'x': x_coords,
        'y': y_coords, 
        'value': values
    })
    df_time = (time.time() - df_start) * 1000
    print(f"✓ DataFrame created: {df_time:.1f} ms")
    
    # Configure canvas for aggregation
    canvas_width, canvas_height = 1024, 1024  # High resolution for quality
    x_range = (x_coords.min(), x_coords.max())
    y_range = (y_coords.min(), y_coords.max())
    
    canvas = ds.Canvas(
        plot_width=canvas_width,
        plot_height=canvas_height,
        x_range=x_range,
        y_range=y_range
    )
    
    print(f"✓ Canvas configured: {canvas_width}×{canvas_height}")
    print(f"   X range: {x_range}")
    print(f"   Y range: {y_range}")
    
    # Aggregate points by mean value
    agg_start = time.time()
    agg = canvas.points(df, 'x', 'y', ds.mean('value'))
    agg_time = (time.time() - agg_start) * 1000
    
    print(f"✓ Aggregation complete: {agg_time:.1f} ms")
    print(f"   Aggregation shape: {agg.shape}")
    print(f"   Value range: [{float(agg.min()):.3f}, {float(agg.max()):.3f}]")
    print(f"   Non-zero pixels: {np.count_nonzero(agg.values):,}")
    
else:
    print("⚠ Skipping datashader aggregation - not available")
    # Create fallback synthetic aggregation
    canvas_width, canvas_height = 512, 512
    agg = None

## forge3d Integration via Adapter

Convert datashader output to forge3d overlay texture using the adapter system.

In [ ]:
if datashader_available and agg is not None:
    print("🔗 forge3d adapter integration...")
    
    # Use adapter for zero-copy conversion
    adapter_start = time.time()
    
    try:
        # Convert aggregation to RGBA overlay using adapter
        extent = (float(x_range[0]), float(y_range[0]), 
                 float(x_range[1]), float(y_range[1]))
        
        overlay_data = shade_to_overlay(
            agg, 
            extent=extent,
            cmap='plasma',  # High contrast for points
            how='log'       # Log scaling for better point visibility
        )
        
        adapter_time = (time.time() - adapter_start) * 1000
        
        print(f"✓ Adapter conversion: {adapter_time:.1f} ms")
        print(f"   RGBA shape: {overlay_data['rgba'].shape}")
        print(f"   Texture format: {overlay_data['format']}")
        print(f"   Memory sharing: {overlay_data['shares_memory']}")
        print(f"   Total bytes: {overlay_data['total_bytes'] / 1024 / 1024:.1f} MB")
        
        # Validate alignment
        alignment_info = validate_alignment(
            extent, None, overlay_data['width'], overlay_data['height']
        )
        print(f"✓ Coordinate alignment validated: {alignment_info['within_tolerance']}")
        
        rgba_overlay = overlay_data['rgba']
        
    except Exception as e:
        print(f"✗ Adapter conversion failed: {e}")
        raise
        
else:
    print("⚠ Creating fallback visualization...")
    # Create simple fallback visualization
    rgba_overlay = np.zeros((512, 512, 4), dtype=np.uint8)
    
    # Simple scatter plot fallback
    for i in range(0, min(10000, len(x_coords)), 100):
        px = int((x_coords[i] - x_coords.min()) / (x_coords.max() - x_coords.min()) * 511)
        py = int((y_coords[i] - y_coords.min()) / (y_coords.max() - y_coords.min()) * 511)
        if 0 <= px < 512 and 0 <= py < 512:
            rgba_overlay[py, px] = [255, 100, 200, 255]  # Magenta points
    
    extent = (float(x_coords.min()), float(y_coords.min()), 
             float(x_coords.max()), float(y_coords.max()))

## GPU Rendering with Overlay

Render the point visualization using forge3d's GPU pipeline.

In [ ]:
# Initialize renderer for overlay rendering
render_start = time.time()

try:
    renderer = f3d.Renderer(1024, 1024, prefer_software=False)
    print(f"✓ Renderer initialized: {renderer.info()}")
    
    # Set up basic scene (we'll overlay points on top)
    # Create simple background terrain
    bg_size = 64
    bg_terrain = np.zeros((bg_size, bg_size), dtype=np.float32)
    renderer.upload_height_r32f(bg_terrain, spacing=10.0, exaggeration=0.1)
    
    # Configure camera to match data extent
    center_x = (extent[0] + extent[2]) / 2
    center_y = (extent[1] + extent[3]) / 2
    range_x = extent[2] - extent[0]
    range_y = extent[3] - extent[1]
    camera_height = max(range_x, range_y) * 0.8
    
    renderer.set_camera(
        eye=(center_x, camera_height, center_y + range_y * 0.3),
        target=(center_x, 0.0, center_y),
        up=(0.0, 1.0, 0.0)
    )
    
    print(f"✓ Camera positioned for extent: {extent}")
    
    # Render base scene
    base_render_start = time.time()
    base_rgba = renderer.render_rgba()
    base_render_time = (time.time() - base_render_start) * 1000
    
    print(f"✓ Base scene rendered: {base_render_time:.1f} ms")
    
except Exception as e:
    print(f"✗ Renderer setup failed: {e}")
    raise

In [ ]:
# Composite overlay with base scene
print("🎨 Compositing overlay...")

composite_start = time.time()

try:
    # Resize overlay to match render target if needed
    target_height, target_width = base_rgba.shape[:2]
    overlay_height, overlay_width = rgba_overlay.shape[:2]
    
    if (overlay_height, overlay_width) != (target_height, target_width):
        print(f"   Resizing overlay: {overlay_width}×{overlay_height} → {target_width}×{target_height}")
        
        # Simple nearest-neighbor resize for demo
        scale_x = target_width / overlay_width
        scale_y = target_height / overlay_height
        
        resized_overlay = np.zeros((target_height, target_width, 4), dtype=np.uint8)
        for y in range(target_height):
            for x in range(target_width):
                src_x = min(int(x / scale_x), overlay_width - 1)
                src_y = min(int(y / scale_y), overlay_height - 1)
                resized_overlay[y, x] = rgba_overlay[src_y, src_x]
        
        rgba_overlay = resized_overlay
    
    # Alpha blend overlay onto base scene
    final_rgba = base_rgba.copy()
    
    # Simple alpha blending where overlay alpha > 0
    overlay_mask = rgba_overlay[:, :, 3] > 0
    alpha = rgba_overlay[:, :, 3:4] / 255.0
    inv_alpha = 1.0 - alpha
    
    # Blend RGB channels
    for c in range(3):
        final_rgba[:, :, c] = (
            final_rgba[:, :, c] * inv_alpha[:, :, 0] +
            rgba_overlay[:, :, c] * alpha[:, :, 0]
        ).astype(np.uint8)
    
    # Keep original alpha from base
    blended_pixels = np.sum(overlay_mask)
    
    composite_time = (time.time() - composite_start) * 1000
    total_render_time = (time.time() - render_start) * 1000
    
    print(f"✓ Overlay composited: {composite_time:.1f} ms")
    print(f"   Blended pixels: {blended_pixels:,}")
    print(f"   Total render time: {total_render_time:.1f} ms")
    
except Exception as e:
    print(f"✗ Overlay composition failed: {e}")
    final_rgba = base_rgba  # Fallback to base scene

## Output Generation & Validation

Save final image and validate output quality.

In [ ]:
# Save output
output_path = "datashader_points.png"
save_start = time.time()

try:
    f3d.numpy_to_png(output_path, final_rgba)
    save_time = (time.time() - save_start) * 1000
    
    # Verify output file
    if os.path.exists(output_path):
        file_size = os.path.getsize(output_path)
        print(f"✓ Output saved: {output_path} ({file_size / 1024:.1f} KB, {save_time:.1f} ms)")
    else:
        raise FileNotFoundError(f"Output file not created: {output_path}")
        
except Exception as e:
    print(f"✗ Save failed: {e}")
    raise

## Performance & Memory Analysis

Analyze the complete pipeline performance and memory usage.

In [ ]:
# Comprehensive performance summary
total_notebook_time = (time.time() - start_time) * 1000

print("📊 Performance Summary:")
print(f"   Points processed: {num_points:,}")
print(f"   Point generation: {generation_time:.1f} ms")
if datashader_available:
    print(f"   DataFrame creation: {df_time:.1f} ms")
    print(f"   Datashader aggregation: {agg_time:.1f} ms")
    print(f"   Adapter conversion: {adapter_time:.1f} ms")
print(f"   GPU base rendering: {base_render_time:.1f} ms")
print(f"   Overlay composition: {composite_time:.1f} ms")
print(f"   PNG encoding: {save_time:.1f} ms")
print(f"   Total notebook time: {total_notebook_time:.1f} ms")

# Throughput calculations
if datashader_available:
    points_per_second = num_points / (total_notebook_time / 1000)
    print(f"   Throughput: {points_per_second:,.0f} points/second")

# Memory analysis
print(f"\n💾 Memory Analysis:")
point_data_mb = points_array.nbytes / (1024 * 1024)
output_mb = final_rgba.nbytes / (1024 * 1024)
overlay_mb = rgba_overlay.nbytes / (1024 * 1024)
total_memory = point_data_mb + output_mb + overlay_mb

print(f"   Point data: {point_data_mb:.1f} MB")
print(f"   Overlay texture: {overlay_mb:.1f} MB")
print(f"   Output image: {output_mb:.1f} MB")
print(f"   Total arrays: {total_memory:.1f} MB")
print(f"   Budget compliance: {'✓' if total_memory <= memory_budget_mb else '⚠'} (<{memory_budget_mb} MB target)")

# Quality metrics
print(f"\n🔍 Output Quality:")
print(f"   Image dimensions: {final_rgba.shape}")
print(f"   Data range: [{final_rgba.min()}, {final_rgba.max()}] {final_rgba.dtype}")

# Content validation
non_zero_rgb = np.count_nonzero(final_rgba[:,:,:3])
total_rgb_values = final_rgba.shape[0] * final_rgba.shape[1] * 3
content_ratio = non_zero_rgb / total_rgb_values

print(f"   Content ratio: {content_ratio:.2%} non-zero RGB values")
print(f"   File size: {file_size / 1024:.1f} KB")

# Runtime compliance
max_runtime_ms = 8 * 60 * 1000  # 8 minutes
runtime_ok = total_notebook_time <= max_runtime_ms

print(f"\n⏱️  Runtime Compliance:")
print(f"   Actual: {total_notebook_time / 1000:.1f}s")
print(f"   Target: <{max_runtime_ms / 1000:.0f}s")
print(f"   Status: {'✓' if runtime_ok else '⚠'} {'Within budget' if runtime_ok else 'Exceeded budget'}")

# Integration status
print(f"\n🔗 Integration Status:")
if datashader_available:
    print(f"   Datashader: ✓ Active")
    print(f"   Zero-copy adapter: ✓ Enabled")
    print(f"   Aggregation method: Mean value with log scaling")
else:
    print(f"   Datashader: ⚠ Fallback mode")
    
print(f"   GPU acceleration: ✓ Active")
print(f"   Overlay composition: ✓ Alpha blending")

print(f"\n✅ Datashader notebook completed!")
print(f"📁 Output: {output_path}")

## Metadata Export

Export performance and configuration metadata for CI validation.

In [ ]:
# Export metadata for CI validation
metadata = {
    "notebook": "datashader_points.ipynb",
    "timestamp": time.time(),
    "performance": {
        "points_processed": int(num_points),
        "total_time_ms": float(total_notebook_time),
        "datashader_available": datashader_available,
        "generation_time_ms": float(generation_time),
        "render_time_ms": float(base_render_time),
        "save_time_ms": float(save_time)
    },
    "memory": {
        "budget_mb": memory_budget_mb,
        "used_mb": float(total_memory),
        "compliance": total_memory <= memory_budget_mb
    },
    "output": {
        "file": output_path,
        "size_kb": float(file_size / 1024),
        "dimensions": list(final_rgba.shape),
        "content_ratio": float(content_ratio)
    },
    "validation": {
        "runtime_ok": runtime_ok,
        "memory_ok": total_memory <= memory_budget_mb,
        "output_created": os.path.exists(output_path),
        "has_content": content_ratio > 0.01
    }
}

if datashader_available:
    metadata["performance"].update({
        "dataframe_time_ms": float(df_time),
        "aggregation_time_ms": float(agg_time),
        "adapter_time_ms": float(adapter_time),
        "points_per_second": float(points_per_second)
    })

# Save metadata
metadata_path = "datashader_points_metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"📋 Metadata exported: {metadata_path}")
print(f"   All validations: {'✅ PASS' if all(metadata['validation'].values()) else '❌ FAIL'}")